In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-img/954471/Train/F/121282643_1739796282838922_3924762867765759060_n.jpg
/kaggle/input/data-img/954471/Train/F/121597221_972711616544097_204099069773999800_o.jpg
/kaggle/input/data-img/954471/Train/F/121469650_1012059645926928_9115015357761950295_o.jpg
/kaggle/input/data-img/954471/Train/F/121453866_1739796116172272_6350458709778971249_n.jpg
/kaggle/input/data-img/954471/Train/F/121256709_972711336544125_7929623606048122247_o.jpg
/kaggle/input/data-img/954471/Train/F/121526847_972713809877211_481376914472549411_o.jpg
/kaggle/input/data-img/954471/Train/F/121460887_1012057685927124_1603928742736483276_o.jpg
/kaggle/input/data-img/954471/Train/F/121465524_1012059792593580_5614978863786312552_o.jpg
/kaggle/input/data-img/954471/Train/M/121222913_2888173951406016_2943121663743613328_n.jpg
/kaggle/input/data-img/954471/Train/M/121335583_2114014788729050_910466790392877863_n.jpg
/kaggle/input/data-img/954471/Train/M/121368482_3314461121964215_543249990768154260_o.jpg
/kaggl

In [4]:
test_dir="/kaggle/input/data-img/954471/Test"
train_dir="/kaggle/input/data-img/954471/Train"

train_dir_F = train_dir + '/F'
train_dir_M = train_dir + '/M'
test_dir_F = test_dir + '/F'
test_dir_M = test_dir + '/M'

In [5]:
print('number of F training images - ',len(os.listdir(train_dir_F)))
print('number of M training images - ',len(os.listdir(train_dir_M)))
print('number of F testing images - ',len(os.listdir(test_dir_F)))
print('number of M testing images - ',len(os.listdir(test_dir_M)))

number of F training images -  8
number of M training images -  8
number of F testing images -  4
number of M testing images -  4


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        rescale = 1.0/255.0,   # Intensity Normalized
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,   # randomly flip images
        validation_split=0.2)

In [7]:
batch_size = 4  #การเอารูปใส่เข้าไป
img_height = 64 
img_width  = 64 
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = data_generator.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

Found 14 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'F': 0, 'M': 1}

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 200)

Epoch 1/200
3/3 [==============================] - 0s 34ms/step - loss: 0.7367 - accuracy: 0.5000
Epoch 2/200
3/3 [==============================] - 0s 23ms/step - loss: 0.6418 - accuracy: 0.7000
Epoch 3/200
3/3 [==============================] - 0s 23ms/step - loss: 0.6796 - accuracy: 0.4000
Epoch 4/200
3/3 [==============================] - 0s 23ms/step - loss: 0.6697 - accuracy: 0.7000
Epoch 5/200
3/3 [==============================] - 0s 28ms/step - loss: 0.7290 - accuracy: 0.4000
Epoch 6/200
3/3 [==============================] - 0s 28ms/step - loss: 0.8076 - accuracy: 0.6000
Epoch 7/200
3/3 [==============================] - 0s 26ms/step - loss: 0.7282 - accuracy: 0.4167
Epoch 8/200
3/3 [==============================] - 0s 23ms/step - loss: 0.7081 - accuracy: 0.5000
Epoch 9/200
3/3 [==============================] - 0s 29ms/step - loss: 0.6502 - accuracy: 0.7000
Epoch 10/200
3/3 [==============================] - 0s 27ms/step - loss: 0.7029 - accuracy: 0.5000
Epoch 11/200
3/3 [=

3/3 [==============================] - 0s 24ms/step - loss: 3.2403e-04 - accuracy: 1.0000
Epoch 84/200
3/3 [==============================] - 0s 29ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 85/200
3/3 [==============================] - 0s 28ms/step - loss: 3.8621e-04 - accuracy: 1.0000
Epoch 86/200
3/3 [==============================] - 0s 20ms/step - loss: 1.2793e-04 - accuracy: 1.0000
Epoch 87/200
3/3 [==============================] - 0s 28ms/step - loss: 8.7352e-04 - accuracy: 1.0000
Epoch 88/200
3/3 [==============================] - 0s 20ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 89/200
3/3 [==============================] - 0s 27ms/step - loss: 4.5942e-04 - accuracy: 1.0000
Epoch 90/200
3/3 [==============================] - 0s 22ms/step - loss: 9.2309e-04 - accuracy: 1.0000
Epoch 91/200
3/3 [==============================] - 0s 25ms/step - loss: 2.8468e-05 - accuracy: 1.0000
Epoch 92/200
3/3 [==============================] - 0s 25ms/step - loss: 2.1944e-04 - accuracy

3/3 [==============================] - 0s 23ms/step - loss: 1.5561e-06 - accuracy: 1.0000
Epoch 163/200
3/3 [==============================] - 0s 24ms/step - loss: 2.5356e-06 - accuracy: 1.0000
Epoch 164/200
3/3 [==============================] - 0s 24ms/step - loss: 1.8891e-05 - accuracy: 1.0000
Epoch 165/200
3/3 [==============================] - 0s 23ms/step - loss: 9.5918e-06 - accuracy: 1.0000
Epoch 166/200
3/3 [==============================] - 0s 28ms/step - loss: 3.4328e-05 - accuracy: 1.0000
Epoch 167/200
3/3 [==============================] - 0s 27ms/step - loss: 5.8776e-04 - accuracy: 1.0000
Epoch 168/200
3/3 [==============================] - 0s 22ms/step - loss: 1.8942e-06 - accuracy: 1.0000
Epoch 169/200
3/3 [==============================] - 0s 22ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 170/200
3/3 [==============================] - 0s 25ms/step - loss: 9.8420e-07 - accuracy: 1.0000
Epoch 171/200
3/3 [==============================] - 0s 23ms/step - loss: 2.2633e-

In [34]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/M/121160480_3678313258869851_5818106878740531607_o.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)

[[0.]]
